In [3]:
import requests
import pandas as pd
from datetime import datetime, timezone

In [4]:
def get_yearly_stats(appid, year):
   
    url = f"https://steamcharts.com/app/{appid}/chart-data.json"
    response = requests.get(url)
    if response.status_code != 200:
        return None, None
    
    data = response.json()  
    
    year_data = []
    for ts, avg_players in data:
        dt = datetime.fromtimestamp(ts / 1000, tz=timezone.utc)  
        if dt.year == year:
            year_data.append(avg_players)
    
    if not year_data:
        return None, None 
    
    peak = max(year_data) # pico do ano
    avg = sum(year_data) / len(year_data)  # média do ano
    return peak, avg


anos = [2019, 2020, 2021, 2022, 2023, 2024]

for ano in anos:
    
    df = pd.read_csv(f"data/most_played_{ano}.csv")
    
    resultados = []
    for appid in df["id"]:
        peak, avg = get_yearly_stats(appid, ano)
        resultados.append({
            "id": appid,
            "peak": peak,
            "average": avg
        })
    
    # salvar em CSV
    df_result = pd.DataFrame(resultados)
    df_result.to_csv(f"data/most_played_{ano}.csv", index=False)